<a href="https://colab.research.google.com/github/yoursmaddyy/Machine-Learning-Track/blob/main/kaggle_road_accident.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from google.colab import files

In [ ]:
df = pd.read_csv("/content/train.csv")

In [ ]:
df.head()

,id,road_type,num_lanes,curvature,speed_limit,lighting,weather,road_signs_present,public_road,time_of_day,holiday,school_season,num_reported_accidents,accident_risk
0,0,urban,2,0.06,35,daylight,rainy,False,True,afternoon,False,True,1,0.13
1,1,urban,4,0.99,35,daylight,clear,True,False,evening,True,True,0,0.35
2,2,rural,4,0.63,70,dim,clear,False,True,morning,True,False,2,0.30
3,3,highway,4,0.07,35,dim,rainy,True,True,morning,False,False,1,0.21
4,4,rural,1,0.58,60,daylight,foggy,False,False,evening,True,False,1,0.56


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 517754 entries, 0 to 517753
Data columns (total 14 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   id                      517754 non-null  int64  
 1   road_type               517754 non-null  object 
 2   num_lanes               517754 non-null  int64  
 3   curvature               517754 non-null  float64
 4   speed_limit             517754 non-null  int64  
 5   lighting                517754 non-null  object 
 6   weather                 517754 non-null  object 
 7   road_signs_present      517754 non-null  bool   
 8   public_road             517754 non-null  bool   
 9   time_of_day             517754 non-null  object 
 10  holiday                 517754 non-null  bool   
 11  school_season           517754 non-null  bool   
 12  num_reported_accidents  517754 non-null  int64  
 13  accident_risk           517754 non-null  float64
dtypes: bool(4), float64(

In [ ]:
df.drop("id", axis=1, inplace=True)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 517754 entries, 0 to 517753
Data columns (total 13 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   road_type               517754 non-null  object 
 1   num_lanes               517754 non-null  int64  
 2   curvature               517754 non-null  float64
 3   speed_limit             517754 non-null  int64  
 4   lighting                517754 non-null  object 
 5   weather                 517754 non-null  object 
 6   road_signs_present      517754 non-null  bool   
 7   public_road             517754 non-null  bool   
 8   time_of_day             517754 non-null  object 
 9   holiday                 517754 non-null  bool   
 10  school_season           517754 non-null  bool   
 11  num_reported_accidents  517754 non-null  int64  
 12  accident_risk           517754 non-null  float64
dtypes: bool(4), float64(2), int64(3), object(4)
memory usage: 37.5+ MB


In [ ]:
from sklearn.preprocessing import LabelEncoder

for col in df.select_dtypes('object'):
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])


In [ ]:
df.head()

,road_type,num_lanes,curvature,speed_limit,lighting,weather,road_signs_present,public_road,time_of_day,holiday,school_season,num_reported_accidents,accident_risk
0,2,2,0.06,35,0,2,False,True,0,False,True,1,0.13
1,2,4,0.99,35,0,0,True,False,1,True,True,0,0.35
2,1,4,0.63,70,1,0,False,True,2,True,False,2,0.30
3,0,4,0.07,35,1,2,True,True,2,False,False,1,0.21
4,1,1,0.58,60,0,1,False,False,1,True,False,1,0.56


In [ ]:
from sklearn.model_selection import train_test_split

X = df.drop('accident_risk', axis=1)
y = df['accident_risk']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
import lightgbm as lgb
from sklearn.metrics import mean_squared_error

train_data = lgb.Dataset(X_train, label=y_train)
test_data = lgb.Dataset(X_test, label=y_test)

params = {
    'objective': 'regression',
    'metric': 'rmse',
    'boosting_type': 'gbdt',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9
}

model = lgb.train(params, train_data, valid_sets=[test_data], num_boost_round=500)


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.017116 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 169
[LightGBM] [Info] Number of data points in the train set: 414203, number of used features: 12
[LightGBM] [Info] Start training from score 0.352605


In [ ]:
train_pred = model.predict(X_train)
test_pred = model.predict(X_test)

In [ ]:
from sklearn.metrics import mean_absolute_error, accuracy_score, r2_score

In [ ]:
print("Accuracy of training :", r2_score(y_train, train_pred))
print("Accuracy of testing :", r2_score(y_test, test_pred))


Accuracy of training : 0.8881877336180304
Accuracy of testing : 0.8852660755325739


In [ ]:
from sklearn.ensemble import RandomForestRegressor


In [ ]:
ran = RandomForestRegressor(
    n_estimators=300,          # number of trees (200–500 usually enough)
    max_depth=20,              # controls overfitting; try 10–30
    min_samples_split=10,      # minimum samples needed to split an internal node
    min_samples_leaf=5,        # minimum samples at a leaf node
    max_features='sqrt',       # use sqrt of features at each split (reduces correlation between trees)
    bootstrap=True,            # random sampling of data for each tree
    n_jobs=-1,                 # use all CPU cores
    random_state=42,           # reproducibility
    verbose=1                  # to monitor progress on large data
)
ran.fit(X_train, y_train)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:   21.2s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  2.3min finished


RandomForestRegressor(max_depth=20, max_features='sqrt', min_samples_leaf=5,
                      min_samples_split=10, n_estimators=300, n_jobs=-1,
                      random_state=42, verbose=1)

In [ ]:
train_pred2 = ran.predict(X_train)
test_pred2 = ran.predict(X_test)

[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    4.8s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:   26.2s
[Parallel(n_jobs=2)]: Done 300 out of 300 | elapsed:   39.1s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    2.4s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    6.4s
[Parallel(n_jobs=2)]: Done 300 out of 300 | elapsed:    9.1s finished


In [ ]:
print("Accuracy of training :", r2_score(y_train, train_pred2))
print("Accuracy of testing :", r2_score(y_test, test_pred2))


Accuracy of training : 0.9119720537914334
Accuracy of testing : 0.8827471693589919


In [ ]:
df2 = pd.read_csv("test.csv")

In [ ]:
for col in df2.select_dtypes('object'):
    le = LabelEncoder()
    df2[col] = le.fit_transform(df2[col])


In [ ]:
ids = df2["id"]

In [ ]:
df2.drop("id", axis=1, inplace=True)

In [ ]:
re = model.predict(df2)

array([0.28949455, 0.1246907 , 0.18698221, ..., 0.18371356, 0.22085112,
       0.13034761])

In [ ]:
results = ids.copy()  # copy your IDs
results.columns = ['id']  # ensure column name is consistent

In [ ]:
results

,id
0,517754
1,517755
2,517756
3,517757
4,517758
...,...
76434,594188
76435,594189
76436,594190
76437,594191


(76439,)

In [ ]:
submission = pd.DataFrame({'id': results, 'accident_risk': re})
display(submission.head())

,id,accident_risk
0,517754,0.289495
1,517755,0.124691
2,517756,0.186982
3,517757,0.320346
4,517758,0.411748


In [ ]:
submission.to_csv("accident_risk_prediction.csv", index=False)


In [ ]:
submission["accident_risk"] = submission["accident_risk"].round(3)


In [ ]:
submission.head()

,id,accident_risk
0,517754,0.289
1,517755,0.125
2,517756,0.187
3,517757,0.320
4,517758,0.412


In [ ]:
submission.shape

(172585, 2)